<a href="https://colab.research.google.com/github/nerfstudio-project/nerfstudio/tree/main/colab/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
    <picture>
    <source media="(prefers-color-scheme: dark)" srcset="https://docs.nerf.studio/en/latest/_images/logo-dark.png">
    <source media="(prefers-color-scheme: light)" srcset="https://docs.nerf.studio/en/latest/_images/logo.png">
    <img alt="nerfstudio" src="https://docs.nerf.studio/en/latest/_images/logo.png" width="400">
    </picture>
</p>


# Nerfstudio: A collaboration friendly studio for NeRFs


![GitHub stars](https://img.shields.io/github/stars/nerfstudio-project/nerfstudio?color=gold&style=social)

This colab shows how to train and view NeRFs from Nerfstudio both on pre-made datasets or from your own videos/images.

\\

Credit to [NeX](https://nex-mpi.github.io/) for Google Colab format.

## Frequently Asked Questions

*  **Downloading custom data is stalling (no output):**
    * This is a bug in Colab. The data is processing, but may take a while to complete. You will know processing completed if `data/nerfstudio/custom_data/transforms.json` exists.
*   **Training is not showing progress**:
    * The lack of output is a bug in Colab. You can see the training progress from the viewer.

* **Viewer Quality is bad / Low resolution**:
    * This may be because more GPU is being used on training that rendering the viewer. Try pausing training or decreasing training utilization.

* **Other problems?**
    * Feel free to create an issue on our [GitHub repo](https://github.com/nerfstudio-project/nerfstudio).


In [ ]:
#@title # Install Conda (requires runtime restart) { vertical-output: true, display-mode: "form" }

!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:23
🔁 Restarting kernel...


In [ ]:
#@title # Install Nerfstudio and Dependencies (~10 min) { vertical-output: true, display-mode: "form" }

%cd /content/
!pip install --upgrade pip
!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 -f https://download.pytorch.org/whl/torch_stable.html

# Installing TinyCuda
%cd /content/
!gdown "https://drive.google.com/u/1/uc?id=1q8fuc-Mqiev5GTBTRA5UPgCaQDzuqKqj" 
!pip install tinycudann-1.6-cp37-cp37m-linux_x86_64.whl

# Installing COLMAP
%cd /content/
!conda install -c conda-forge colmap

# Install nerfstudio
%cd /content/
!pip install nerfstudio

In [ ]:
#@markdown <h1> Downloading Data</h1>
#@markdown <h3>Pick the preset scene or upload your own images/video</h3>
import os
from google.colab import files
from IPython.core.display import display, HTML

scene = '\uD83D\uDDBC poster' #@param ['🖼 poster', '🚜 dozer', '🌄 desolation', '📤 upload your images' , '🎥 upload your own video']
scene = ' '.join(scene.split(' ')[1:])

if scene not in ['upload your images', 'upload your own video']:
    %cd /content/
    !ns-download-data --dataset=nerfstudio --capture=$scene
else:
    display(HTML('<h3>Select your custom data</h3>'))
    display(HTML('<p/>You can select multiple images by pressing ctrl, cmd or shift and click.<p>'))
    display(HTML('<p/>Note: This may take time, especially on hires inputs, so we recommend to download dataset after creation.<p>'))
    !mkdir -p /content/data/nerfstudio/custom_data
    if scene == 'upload your images':
        !mkdir -p /content/data/nerfstudio/custom_data/raw_images
        %cd /content/data/nerfstudio/custom_data/raw_images
        uploaded = files.upload()
        dir = os.getcwd()
    else:
        %cd /content/data/nerfstudio/custom_data/
        uploaded = files.upload()
        dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    del uploaded
    %cd /content/

    if scene == 'upload your images':
        !ns-process-data images --data /content/data/nerfstudio/custom_data/raw_images --output-dir /content/data/nerfstudio/custom_data/
    else:
        video_path = preupload_datasets[0]
        !ns-process-data video --data $video_path --output-dir /content/data/nerfstudio/custom_data/

    scene = "custom_data"

/content
Downloading...
From: https://drive.google.com/uc?id=1dmjWGXlJnUxwosN6MVooCDQe970PkD-1
To: /content/data/nerfstudio/poster.zip
100% 750M/750M [00:03<00:00, 218MB/s]


In [ ]:
#@title # Set Up Viewer

%cd /content

# Install localtunnel
# We are using localtunnel https://github.com/localtunnel/localtunnel but ngrok could also be used
!npm install -g localtunnel

# Tunnel port 7007, the default for
!rm url.txt 2> /dev/null
get_ipython().system_raw('lt --port 7007 >> url.txt 2>&1 &')

/content
/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 1.536s


In [ ]:
#@title # Start Viewer

with open('url.txt') as f:
  lines = f.readlines()
websocket_url = lines[0].split(": ")[1].strip().replace("https", "wss")
# from nerfstudio.utils.io import load_from_json
# from pathlib import Path
# json_filename = "nerfstudio/nerfstudio/viewer/app/package.json"
# version = load_from_json(Path(json_filename))["version"]
url = f"https://viewer.nerf.studio/?websocket_url={websocket_url}"
print(url)
print("You may need to click Refresh Page after you start training!")
from IPython import display
display.IFrame(src=url, height=800, width="100%")

https://viewer.nerf.studio/?websocket_url=wss://good-ways-ask-34-83-255-230.loca.lt
You may need to click Refresh Page after you start training!


In [ ]:
#@title # Start Training { vertical-output: true }

%cd /content
!ns-train nerfacto --viewer.websocket-port 7007 nerfstudio-data --data data/nerfstudio/$scene --downscale-factor 4

In [ ]:
#@title # Render Video { vertical-output: true }
#@markdown <h3>Export the camera path from within the viewer, then run this cell.</h3>
#@markdown <h5>The rendered video should be at renders/output.mp4!</h5>


base_dir = "/content/outputs/data-nerfstudio-" + scene + "/nerfacto/"
training_run_dir = base_dir + os.listdir(base_dir)[0]

from IPython.core.display import display, HTML
display(HTML('<h3>Upload the camera path JSON.</h3>'))
%cd $training_run_dir
uploaded = files.upload()
uploaded_camera_path_filename = list(uploaded.keys())[0]

config_filename = training_run_dir + "/config.yml"
camera_path_filename = training_run_dir + "/" + uploaded_camera_path_filename
camera_path_filename = camera_path_filename.replace(" ", "\\ ").replace("(", "\\(").replace(")", "\\)")

%cd /content/
!ns-render --load-config $config_filename --traj filename --camera-path-filename $camera_path_filename --output-path renders/output.mp4

[19:48:48] Skipping 0 files in dataset split train.                                          ]8;id=527413;file:///content/nerfstudio/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=243595;file:///content/nerfstudio/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#91\91]8;;\
           Skipping 0 files in dataset split test.                                           ]8;id=109270;file:///content/nerfstudio/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=464675;file:///content/nerfstudio/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#91\91]8;;\
Loading data batch ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
/usr/local/lib/python3.7/site-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Plea